<a href="https://colab.research.google.com/github/yaroslavtsepkov/APC/blob/main/genetic_alg/genetic_alg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
import time
import pandas as pd

random.seed(0)

# genetic alg with numpy

## parametrs

In [ ]:
num_weight = 5 
weight = np.random.uniform(low=-1.0, high=1.0, size=num_weight) 
individum = 1000
pop_size = (individum, num_weight)
new_pop = np.random.uniform(low=-1.0, high=1.0, size=pop_size)
max_gen = 2000
num_parents = 8

## visual

In [ ]:
x = np.linspace(0, 10, 500)
y = np.array([
    weight[0] * (x**4) + 
    weight[1] * (x**3) + 
    weight[2] * (x**2) + 
    weight[3] * x +
    weight[4] for x in x         
])

fig, ax = plt.subplots(figsize = (12, 6))
ax.plot(x, y, label='true', color='#ed8087', linewidth=3)
ax.grid(True)
ax.legend(fontsize=14)

fig.show()

## fitness func

In [ ]:
np.array(new_pop).shape

In [ ]:
def fitnessGA(weight, pop):
    fitness = []
    for p in pop:
        fitness.append(np.sum([np.abs(weight[i] - p[i]) / (np.abs(weight[i]) + 1e-6) for i in range(weight.shape[0])]))
    return np.array(fitness)

In [ ]:
def fitnessGA(weight, pop):
    fitness = []
    for p in pop:
        fitness.append(np.sqrt(((weight - p) ** 2).mean()))
    return fitness

In [ ]:
t = time.time()
fitness = fitnessGA(weight, new_pop)
print('time', time.time() - t)

In [ ]:
fitness

## parents func

In [ ]:
def matingPoolGA(pop, fitness, num_parents):
    parents = np.empty((num_parents, pop.shape[1]))
    first = True
    for parent_num in range(num_parents):
        max_fitness_idx = np.argmin(fitness)
        parents[parent_num, :] = pop[max_fitness_idx, :]
        fitness[max_fitness_idx] = 99999999999
    return parents

## crossover func

In [ ]:
def crossoverGA(parents, offspring_size):
    offspring = np.empty(offspring_size)
    crossover_point = np.uint8(offspring_size[1]/2)

    for k in range(offspring_size[0]):
        parent1_idx = k % parents.shape[0]
        parent2_idx = (k + 1) % parents.shape[0]
        offspring[k, :crossover_point] = parents[parent1_idx, :crossover_point]
        offspring[k, crossover_point:] = parents[parent2_idx, crossover_point:]

    return offspring

## mutation func

In [ ]:
def mutationGA(offspring_crossover):
    for idx in range(offspring_crossover.shape[0]):
        random_value = np.random.uniform(-1.0, 1.0, 1)
        offspring_crossover[idx, 2] = offspring_crossover[idx, 2] + random_value

    return offspring_crossover

## example

In [ ]:
data_time = [] 
data_cpu = pd.DataFrame()
t = time.time()
for _ in range(max_gen):
    t1 = time.time()
    fitness = fitnessGA(weight, new_pop)
    parents = matingPoolGA(new_pop, fitness, num_parents)
    offspring_crossover = crossoverGA(parents, offspring_size=(pop_size[0] - parents.shape[0], num_weight))
    offspring_mutation = mutationGA(offspring_crossover)
    new_pop[:parents.shape[0], :] = parents
    new_pop[parents.shape[0]:, :] = offspring_mutation
time_cpu = round(time.time() - t, 2)

## visual true and predict polynom

In [ ]:
best_weight = new_pop[np.argmin(fitnessGA(weight, new_pop))]
y_pred = np.array([
    best_weight[0] * (x**4) + 
    best_weight[1] * (x**3) + 
    best_weight[2] * (x**2) + 
    best_weight[3] * x +
    best_weight[4] for x in x         
])

fig, ax = plt.subplots(figsize = (12, 6))
ax.plot(x, y, label='true', color='#ed8087', linewidth=3)
ax.plot(x, y_pred, label='predicted', color='#d036ff', linewidth=3)
ax.grid(True)
ax.legend(fontsize=14)

fig.show()

## table best weight and true weight

In [ ]:
data = pd.DataFrame(data={
    'best_weight': best_weight,
    'true_weight': weight
})

In [ ]:
data

## time statics

In [ ]:
df = pd.DataFrame(data={
    'CPU':{
        'time':time_cpu,
        'gen':max_gen
    }
})

In [ ]:
df

# genetic alg with cupy 

## import library

In [ ]:
!curl https://colab.chainer.org/install | sh -

In [ ]:
import chainer
chainer.print_runtime_info()

In [ ]:
import cupy as cp

## parametrs

In [ ]:
num_weight = 5 
weight = cp.random.uniform(low=-1.0, high=1.0, size=num_weight) 
individum = cp.random.randint(500,2000)
pop_size = (individum, num_weight)
new_pop_gpu = cp.array(new_pop)
max_gen = np.random.randint(low=2000,high=5000)
num_parents = 8

In [ ]:
equation_inputs=cp.arange(-10,10,20/1000)
noise = cp.random.uniform(equation_inputs.min()/5, equation_inputs.max()/5, (equation_inputs.shape[0], 1)).T
equation_outputs=-equation_inputs**2+1+noise
#ga_numpy.plot_res(equation_inputs,equation_outputs,equation_outputs)
# Количество коэффициентов, которые мы хотим оптимизировать.
num_weights = 5

"""
Genetic algorithm parameters:
    Mating pool size
    Population size
"""
sol_per_pop = 2000
num_parents_mating = 500

# Определение численности поколения.
pop_size = (sol_per_pop,num_weights) # У населения будет хромосома sol_per_pop, где каждая хромосома имеет num_weights генов.
#Создание начальной популяции. #Изменить
const = 1
new_population = cp.random.uniform(low=-const, high=const, size=pop_size)
print(new_population)

num_generations = 20

## fitness func cupy

In [ ]:
def fitnessGaCupy(weight, pop):
    fitness = []
    tmp = []
    for p in pop:
        for i in range(weight.shape[0]):
            tmp = cp.abs(weight[i] - p[i]) / (cp.abs(weight[i]) + 1e-6) 
            sum = cp.sum(cp.array(tmp))
        fitness.append(sum)
    return cp.array(fitness)

In [ ]:
def cal_pop_fitness(equation_inputs, equation_outputs,pop):
    # Вычисление значения пригодности каждого решения в текущей популяции.
    # Матрица для умножения
    matrix_for_product = cp.ones((pop.shape[1], equation_inputs.shape[0]))
    matrix_for_product[1, :] = copy.deepcopy(equation_inputs)
    matrix_for_product[2, :] = equation_inputs ** 2
    if (matrix_for_product.shape[0]>3):
        matrix_for_product[3, :] = equation_inputs ** 3
        if (matrix_for_product.shape[0] > 4):
            matrix_for_product[4, :] = equation_inputs ** 4
    massiv_of_y = cp.dot(pop, matrix_for_product)
    # Разность
    dist = equation_outputs - massiv_of_y
    # Функция пригодности вычисляет сумму продуктов между каждым входом и соответствующим ему весом.
    fitness = cp.abs(dist).max(axis=1)
    return fitness

## parents func cupy

In [ ]:
def matingPoolGaCupy(pop, fitness, num_parents):
    parents = cp.empty((num_parents, pop.shape[1]))
    first = True
    for parent_num in range(num_parents):
        max_fitness_idx = cp.argmin(fitness)
        parents[parent_num, :] = pop[max_fitness_idx, :]
        fitness[max_fitness_idx] = 99999999999
    return parents

In [ ]:
def matingPoolGaCupy(pop, fitness, num_parents):
    # Выбор лучших особей текущего поколения в качестве родителей для производства потомства следующего поколения.
    parents = cp.empty((num_parents, pop.shape[1]))
    for parent_num in range(num_parents):
        min_fitness_idx = cp.where(fitness == cp.min(fitness))
        min_fitness_idx = min_fitness_idx[0][0]
        parents[parent_num, :] = pop[min_fitness_idx, :]
        fitness[min_fitness_idx] = 99999999999
    return parents

## crossover func cupy

In [ ]:
def crossoverGaCupy(parents, offspring_size):
    offspring = cp.empty(offspring_size)
    crossover_point = cp.uint8(offspring_size[1]/2)

    for k in range(offspring_size[0]):
        parent1_idx = k % parents.shape[0]
        parent2_idx = (k + 1) % parents.shape[0]
        offspring[k, :crossover_point] = parents[parent1_idx, :crossover_point]
        offspring[k, crossover_point:] = parents[parent2_idx, crossover_point:]

    return cp.array(offspring)

In [ ]:
def crossoverGaCupy(parents, offspring_size):
    offspring = cp.empty(offspring_size)
    # Точка, в которой происходит кроссовер между двумя родителями. Обычно это в центре.
    crossover_point = cp.uint8(offspring_size[1]/2)

    for k in range(offspring_size[0]):
        # Индекс первого скрещенного родителя.
        parent1_idx = k%parents.shape[0]
        # Индекс второго родителя для скрещивания.
        parent2_idx = (k+1)%parents.shape[0]
        # У нового потомства первая половина генов будет заимствована у первого родителя.
        offspring[k, 0:crossover_point] = parents[parent1_idx, 0:crossover_point]
        # У нового потомства вторая половина генов будет взята от второго родителя.
        offspring[k, crossover_point:] = parents[parent2_idx, crossover_point:]
    return offspring


## mutation func cupy

In [ ]:
def mutationGACupy(offspring_crossover):
    for idx in range(offspring_crossover.shape[0]):
        random_value = cp.random.uniform(-1.0, 1.0, 1)
        offspring_crossover[idx, 2] = offspring_crossover[idx, 2] + random_value

    return offspring_crossover

## examaple

In [ ]:
tic = time.time()
for generation in range(num_generations):
    print("Generation : ", generation)
    # Измерение приспособленности каждой хромосомы в популяции.
    fitness = fitnessGaCupy(equation_inputs, equation_outputs, new_population)

    # Выбор лучших родителей в популяции для вязки.
    parents = matingPoolGaCupy(new_population, fitness,
                                      num_parents_mating)

    # Создание нового поколения с использованием кроссовера.
    offspring_crossover = crossoverGaCupy(parents,
                                       offspring_size=(pop_size[0]-parents.shape[0], num_weights))

    # Добавление некоторых вариаций потомству с помощью мутации.
    offspring_mutation = mutationGACupy(offspring_crossover,const)

    # Создание новой популяции на основе родителей и потомства.
    new_population[0:parents.shape[0], :] = parents
    new_population[parents.shape[0]:, :] = offspring_mutation

    # Лучший результат в текущей итерации.#Изменить
    # Матрица для умножения
    matrix_for_product = cp.ones((new_population.shape[1], equation_inputs.shape[0]))
    matrix_for_product[1, :] = copy.deepcopy(equation_inputs)
    matrix_for_product[2, :] = equation_inputs ** 2
    if (matrix_for_product.shape[0]>3):
        matrix_for_product[3, :] = equation_inputs ** 3
        if (matrix_for_product.shape[0] > 4):
            matrix_for_product[4, :] = equation_inputs ** 4
    massiv_of_y = cp.dot(new_population, matrix_for_product)
    # Разность
    dist = equation_outputs - massiv_of_y
    # Функция пригодности вычисляет сумму продуктов между каждым входом и соответствующим ему весом.
    fitness_print = cp.abs(dist).max(axis=1)
    print("Лучший результат : ", cp.min(fitness_print))

# Получение лучшего решения после итераций, завершающих все поколения.
#Сначала рассчитывается пригодность для каждого решения в последнем поколении.
fitness = ga_cupy.cal_pop_fitness(equation_inputs, equation_outputs, new_population)
# Затем верните индекс этого решения, соответствующего наилучшей пригодности.
best_match_idx = cp.where(fitness == cp.min(fitness))
toc = time.time()
print('Затрачиваемое время cupy: %f с.' % (toc - tic))
# Матрица для умножения
matrix_for_product = cp.ones((new_population.shape[1], equation_inputs.shape[0]))
matrix_for_product[1, :] = copy.deepcopy(equation_inputs)
matrix_for_product[2, :] = equation_inputs ** 2
if (matrix_for_product.shape[0] > 3):
    matrix_for_product[3, :] = equation_inputs ** 3
    if (matrix_for_product.shape[0] > 4):
        matrix_for_product[4, :] = equation_inputs ** 4
coefs_final = new_population[best_match_idx[0], :].reshape(1,new_population.shape[1])
massiv_of_y1 = cp.dot(coefs_final, matrix_for_product).reshape(equation_inputs.shape[0],)
equation_inputs_n=cp.asnumpy(equation_inputs)
massiv_of_y1_n=cp.asnumpy(massiv_of_y1)
equation_outputs_n=cp.asnumpy(massiv_of_y1)
ga_cupy.plot_res(equation_inputs_n,massiv_of_y1_n,equation_outputs_n)

print("Лучший результат : ", new_population[best_match_idx[0], :])
print("Лучшее значения приспособленности : ", fitness[best_match_idx[0]])

## visual true and predict polynom

In [ ]:
best_weight = new_pop[np.argmin(fitnessGA(weight, new_pop))]
y_pred = np.array([
    best_weight[0] * (x**4) + 
    best_weight[1] * (x**3) + 
    best_weight[2] * (x**2) + 
    best_weight[3] * x +
    best_weight[4] for x in x         
])

fig, ax = plt.subplots(figsize = (12, 6))
ax.plot(x, y, label='true', color='#ed8087', linewidth=3)
ax.plot(x, y_pred, label='predicted', color='#d036ff', linewidth=3)
ax.grid(True)
ax.legend(fontsize=14)

fig.show()

In [ ]:
df = pd.DataFrame(data={
    'CPU':{
        'time':time_gpu,
        'gen':max_gen
    }
})

# genetic alg with pyCUDA

In [ ]:
!pip install pycuda

In [ ]:
!pip install scikit-cuda

In [ ]:
from pycuda import driver, compiler, gpuarray, tools
import pycuda.autoinit
from pycuda.tools import make_default_context
from pycuda.curandom import rand as curand
from pycuda import cumath
from skcuda import linalg, misc
linalg.init()
misc.init()
import xgboost

In [ ]:
import torch
import torch.optim as optim
import torch.nn as nn
#from torchviz import make_dot

device = 'cuda' if torch.cuda.is_available() else 'cpu'


In [ ]:
num_weight = 5 
weight =  torch.from_numpy(np.random.uniform(low=-1.0, high=1.0, size=num_weight)).float().to(device)
individum = 1000
pop_size = (individum, num_weight)
pop = torch.from_numpy(np.random.uniform(low=-1.0, high=1.0, size=pop_size)).float().to(device)
max_gen = 2000
num_parents = 8

In [ ]:
%time gpuarray.sum(pop)

In [ ]:
%time misc.sum(pop)

In [ ]:
 / (np.abs(weight[i]) + 1e-6) for i in range(weight.shape[0])]))

In [ ]:
def fitnessSkCUDA(weight, pop):
    loss = nn.MSELoss()
    return loss(pop, weight)

In [ ]:
t = time.time()
fitness = fitnessSkCUDA(weight, pop)
print('time', time.time() - t)

In [ ]:
fitness.shape